*ToDo*

02) pred_window und obs_window ausprobieren.
03) output nochmal genau anschauen.
04) was ist maskiert anschauen - und wie genau?
05) VRAM wenig ausgelastet. Batch size mal mit 320 ausprobieren, aber auch durchziehen.
06) Task pred: scheduler einfügen.
07) Analyse each var (sparsity)
08) loss per var/quality of varis loss (is this already ablation to forecast only one var)
09) Include sepsis definition
10) 

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 
import joblib
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
def set_seed(seed: int) -> None:
    """
    Set the random seed for modules torch, numpy and random.

    :param seed: random seed
    """
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available() and torch.cuda.device_count() > 0:
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed_all(seed)
set_seed(42)

## Load forecast dataset into matrices.

In [3]:
fore_test_op_dense = joblib.load(open('/home/mitarb/fracarolli/eicu/final/sepsis_label/fore_test_op_susinf.pkl', 'rb'))
fore_test_ip_dense = joblib.load(open('/home/mitarb/fracarolli/eicu/final/sepsis_label/fore_test_ip_susinf.pkl', 'rb'))
test_input = fore_test_op_dense
test_ip = fore_test_ip_dense

mean_std_dict = joblib.load(open('/home/mitarb/fracarolli/eicu/final/mean_std_dict.pkl', 'rb'))
var_to_ind = dict()
with open("/home/mitarb/fracarolli/eicu/final/var_to_ind.csv") as file:
    for line in file:
        key, value = line.strip().split(",")
        var_to_ind[key]=int(value)

In [4]:
print(test_input.shape)
print(len(test_ip[0]))
#fore_test_op_triplet = joblib.load(open('/home/mitarb/fracarolli/eicu/final/evaluation_scripts+pkl/fore_test_op_dense.pkl', 'rb'))
#fore_test_ip_triplet = joblib.load(open('fore_test_ip_dense.pkl', 'rb'))

(3789, 48, 196)
3789


In [5]:
def get_sofa(matrix, var_to_ind): #24xV matrix
    # GCS: min_eye, min_motor, min_verbal = 5, 5, 5
    print(matrix.size())
    #raise Exception
    key ="GCS eye"
    var_to_ind = {x:i-1 for x,i in var_to_ind.items()}
    a=matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]
    print(a)
    min_eye = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=4)
    key = "GCS motor"
    min_motor = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=6)
    key = "GCS verbal"
    min_verbal = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=5)
    

    GCS = min_eye + min_motor + min_verbal
    if GCS > 14: GCS_sofa = 0
    elif GCS > 12: GCS_sofa = 1
    elif GCS > 9:  GCS_sofa = 2
    elif GCS > 5:  GCS_sofa = 3
    else: GCS_sofa = 4
    #print('GCS_sofa is', GCS_sofa, ';     GCS is', GCS,'; GCS eye', min_eye, '; GCS motor', min_motor, '; GCS verbal', min_verbal)

    key = "Bilirubin (Total)"
    bilir = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    if bilir > 12: bilir_sofa = 4
    elif bilir > 6: bilir_sofa = 3
    elif bilir > 2: bilir_sofa = 2
    elif bilir > 1.2: bilir_sofa = 1
    else: bilir_sofa = 0
    #print('bilir_Sofa is', bilir_sofa, ';   bilirubin is', bilir)
    
    # Coagulation (Platelets)
    key = "Platelets"
    plate = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=160)
    if plate > 150: plate_sofa = 0
    elif plate > 100: plate_sofa = 1
    elif plate > 50: plate_sofa = 2
    elif plate > 20: plate_sofa = 3
    else: plate_sofa = 4
    #print('plate_sofa is', plate_sofa, ';   platelet count is', plate)
    
    # print('Urinmenge 24h', sum(data_var[data_var['variable']=='Urine']['value2']))

    key = "Urine"
    urine = sum(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0])
    key = "Creatinine (Blood)"
    creat = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    
    if (urine < 200) or (creat > 5): renal_sofa = 4
    elif  (urine < 500) or (creat > 3.5): renal_sofa = 3
    elif creat > 2.0: renal_sofa = 2
    elif creat > 1.2: renal_sofa = 1
    else: renal_sofa = 0
    #print('renal_sofa:',renal_sofa,';       urine 24:',urine,'; creat:', creat)
    
    CS_data = get_CS(matrix, var_to_ind)
    cs_sofa = CS_SOFA(CS_data)
    
    #cs_sofa = 0
    key="FiO2"
    fio2 = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])
    key="PaO2"
    po2 = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])
    PaO2FiO2 = 100*po2/fio2
    print("size", PaO2FiO2.size())
    PaO2FiO2 = PaO2FiO2[torch.nonzero(PaO2FiO2, as_tuple=True)]
    pao2fio2 = min(PaO2FiO2)
    if pao2fio2<100: resp=4
    elif pao2fio2<200: resp=3
    elif pao2fio2<300:resp=2
    elif pao2fio2<400:resp=1
    else: resp=0
    return GCS_sofa, cs_sofa, resp, plate_sofa, bilir_sofa, renal_sofa

def get_CS(matrix, var_to_ind):
    #data_var = data_pat[data_pat['variable'].isin(['Dobutamine','Dopamine','Epinephrine','Norepinephrine','Weight'])]
    #data_var['value2'] = data_var['value']*data_var['std']+data_var['mean']
    
    #weight = min(data_var[data_var['variable']=='Weight']['value2'], default=80)  # set default weight to 80kg.
    key = "Weight"
    weight = 80# min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)], default=80)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    key = "d Dopamine ratio"
    try:
        data_dop = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_dop = data_dop /60/weight*1000
    except:
        data_dop = 0

    key = "d Dobutamine ratio"
    
    try:
        data_dobu = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_dobu = data_dobu  /60/weight*1000
    except:
        data_dobu = 0
    key = "d Epinephrine ratio"
    
    try:
        data_epi = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_epi = data_epi  /60/weight*1000
    except:
        data_epi = 0
    key = "d Norepinephrine ratio"
    
    try:
        data_nore = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_nore = data_nore /60/weight*1000
    except:
        data_nore = 0
        


    key = "SBP"
    SBP = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])

    key = "DBP"
    DBP = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])

    MAP = 2/3 * DBP + 1/3 * SBP
    MAP = min(MAP[torch.nonzero(MAP, as_tuple=True)], default=100)
                 
        
    return MAP, data_dop, data_dobu, data_epi, data_nore 
    
def CS_SOFA(data):
    map = data[0]
    dop, dobu, epi, nore = data[1:5]
    # print('CS data: mdden', data)
    if (dop > 15) or (epi > 0.1) or (nore > 0.01): CS = 4
    elif (dop > 5) or (epi > 0) or (nore > 0): CS = 3
    elif (dop > 0) or (dobu > 0): CS = 2
    elif map < 70: CS = 1
    else: CS = 0
    # print('CS Sofa is:', CS)
    return CS 
    
factor_keys =["GCS eye", "GCS motor", "GCS verbal", "Bilirubin (Total)", "Platelets", "Urine", "Creatinine (Blood)", "FiO2", "PaO2", "d Dopamine ratio", "d Dobutamine ratio",
"d Epinephrine ratio", "d Norepinephrine ratio", "SBP", "DBP"]
factor_indices = sorted([var_to_ind[x]-1 for x in factor_keys])

In [6]:
d_model, e_layers, d_layers, d_ff = 256, 2, 2, 2048
number_of_epochs = 100
V, D = 98, 17
fore_max_len = 2640
print('loading of joblib completed')



#Michi: die ganzen Argparses könnte man noch anders implementieren, wollte es nur schnell zum laufen bringen.
import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 
#Michi: bisher nur default=3 zum laufen gebracht
parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=V*2, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size')
parser.add_argument('--d_model', type=int, default=d_model, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=e_layers, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=d_layers, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=d_ff, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')
args = parser.parse_args(args=[])
import importlib


import matplotlib.pyplot as plt

loading of joblib completed


In [7]:
import models.InformerAutoregressiveWithForcedDecoding as autoformer
importlib.reload(autoformer)
model = autoformer.Model(args).cuda()


# Crossinteractions

In [8]:
"""
torch.cuda.empty_cache()
import models.InformerAutoregressiveWithForcedDecoding as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()


# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("mimic_iii_24h_strats_no_interp_with_ss_fore_informer_ims_fr_detach.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0

keys = ["Dopamine", "Dobutamine", "Norepinephrine"]

#data_dop = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

min_dict = dict()
max_dict = dict()
twentyfive_dict = dict()
median_dict = dict()
seventyfive_dict = dict()
matrix = torch.tensor(test_input)
for x in range(0, len(test_input)):

    for key in keys:
        try:
            min_data = min(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)])#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
            if key not in min_dict:
                min_dict[key] = min_data
            else:
                min_dict[key] = min([min_data, min_dict[key]])
        except:
            pass
        try:
            max_data = max(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)])#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in max_dict:
                max_dict[key] = max_data
            else:
                max_dict[key] = max([max_data, max_dict[key]])
        except:
            pass
        try:
            twentyfive_data = torch.quantile(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)], 0.25)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in twentyfive_dict:
                twentyfive_dict[key] = twentyfive_data.item()
            else:
                twentyfive_dict[key] = max([twentyfive_data, twentyfive_dict[key]])
        except:
            pass
        try:
            median_data = torch.quantile(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)], 0.50)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in median_dict:
                median_dict[key] = median_data.item()
            else:
                median_dict[key] = max([median_data, median_dict[key]])
        except:
            pass
        try:
            seventyfive_data = torch.quantile(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)], 0.75)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in seventyfive_dict:
                seventyfive_dict[key] = seventyfive_data.item()
            else:
                seventyfive_dict[key] = max([seventyfive_data, seventyfive_dict[key]])
        except:
            pass
print(min_dict, max_dict)
resultdict = dict()
for key in keys:
    maxes, mins, twentyfives, medians, seventyfives = [], [], [], [], []
    for start in pbar:
        print(start)
        matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
        input_matrix = matrix[:, :24]

        #torch.Size([32, 24, 129])
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        input_mask = input_matrix[:, :24, V:]
        output_matrix = matrix[:, 24:, :V]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        output_mask = matrix[:, 24:, V:]
        #torch.Size([32, 24, 129])
        #print(var_to_ind[key])
        #print(min_dict[key])
        #print(max_dict[key])
        with torch.no_grad():
            output_min = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: min_dict[key]})
            output_max = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: max_dict[key]})
            
            output_25 = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: twentyfive_dict[key]})
            output_50 = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: median_dict[key]})
            output_75 = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: seventyfive_dict[key]})
        #print(output1, output2, output1==output2)
        for x in output_min:
            mins.append(x.cpu())
        for x in output_max:
            maxes.append(x.cpu())
        for x in output_25:
            twentyfives.append(x.cpu())
        for x in output_50:
            medians.append(x.cpu())
        for x in output_75:
            seventyfives.append(x.cpu())
    print(len(maxes), maxes[0].size())
    resultdict[(key, "min")]=mins
    resultdict[(key, "max")]=maxes
    resultdict[(key, "25")]=twentyfives
    resultdict[(key, "50")]=medians
    resultdict[(key, "75")]=seventyfives
import pickle

with open('crossinteractions.pickle', 'wb') as handle:
    pickle.dump([resultdict, var_to_ind, min_dict, max_dict, mean_std_dict], handle)
"""

'\ntorch.cuda.empty_cache()\nimport models.InformerAutoregressiveWithForcedDecoding as autoformer\nimportlib.reload(autoformer)\nbatch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6\nd, N, he, dropout = 50, 2, 4, 0.2\nmodel = autoformer.Model(args).cuda()\n\n\n# Load pretrained weights here.\noptimizer = torch.optim.Adam(model.parameters(), lr=lr)\n#fore_savepath = \'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch\'\nscheduler = ReduceLROnPlateau(optimizer, \'max\', patience=3)\nmodel.load_state_dict(torch.load("mimic_iii_24h_strats_no_interp_with_ss_fore_informer_ims_fr_detach.pytorch"))\n#model.load_state_dict(torch.load(fore_savepath))\nmodel.eval()\ny_true, y_pred = [], []\nprint(test_input)\npbar = tqdm(range(0, len(test_input), batch_size))\nloss_list = []\ntest_loss = 0\n\nkeys = ["Dopamine", "Dobutamine", "Norepinephrine"]\n\n#data_dop = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_st

# 8 Sofa Analysis IMS Informer TF

# 7 Sofa Analysis DMS Informer

# 10 Sofa Analysis IMS Informer Backprop

In [14]:

import models.InformerAutoregressiveFullRegression as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("informer_regression_eicu.pytorch"))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
    loss = 0
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        print(output.size())
        a = outpu.item()
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        
        prediction_sofa_difference = (a-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        print(a, sum(b))
        loss_term = (outpu.squeeze()-sum(b))**2
        loss+=loss_term
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FN += 1
        else:
            if prediction_sofa_difference: FP +=1
            else: TN += 1
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()

    test_loss+=loss

print(TP, FP, FN, TN)
print()
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
print(accuracy_sepsis)

[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    1.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.   

  0%|          | 0/119 [00:00<?, ?it/s]

0
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([ 0.5936,  0.5936,  0.5936, -0.4517, -0.4517, -0.4517,  0.5936],
       device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-0.4517,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936], device='cuda:0')
size torch.Size([24])
6.780041694641113 8
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([ 0.5936,  0.5936,  0.5936,  0.5936, -0.4517, -0.4517, -0.4517, -0.4517,
        -0.4517, -0.4517, -0.4517, -0.4517, -0.4517, -0.4517, -0.4517,  0.5936,
        -0.4517, -0.4517, -0.4517, -0.4517, -0.4517, -0.4517], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-0.4517, -0.4517, -0.4517,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,
         0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,
         0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936],
       device='cuda:0')
size torch.Size([24])
5.940263748168945 7
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([], de

  1%|          | 1/119 [00:00<00:52,  2.26it/s]

size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936,
        0.5936, 0.5936], device='cuda:0')
size torch.Size([24])
6.037741661071777 5
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([ 0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936, -0.4517,  0.5936],
       device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936,
        0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936],
       device='cuda:0')
size torch.Size([24])
11.168291091918945 12
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936],
       device='cuda:0')
size torch.Size([24])
11.345685958862305 10
32
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([], device='cuda:0')


...

torch.Size([24, 196])
tensor([-1.4970], device='cuda:0')
size torch.Size([24])
6.09229040145874 4
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
6.084918022155762 6
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
5.479596138000488 4
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-2.5423], device='cuda:0')
size torch.Size([24])
7.095733642578125 4
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([0.5936, 0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936, 0.5936], device='cuda:0')
size torch.Size([24])
5.1018290519714355 4
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
torc

 98%|█████████▊| 117/119 [00:48<00:00,  2.77it/s]

size torch.Size([24])
5.268438816070557 4
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
10.75804615020752 10
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
11.018448829650879 11
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
6.27223539352417 5
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([-2.5423, -2.5423, -2.5423, -2.5423, -2.5423, -2.5423], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([ 0.5936, -2.5423, -2.5423], device='cuda:0')
size torch.Size([24])
9.293039321899414 11
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([-0.4517, -0.4517, -0.4517], device='cuda:0')
size torch.Si

100%|██████████| 119/119 [00:49<00:00,  2.41it/s]

tensor([0.5936], device='cuda:0')
size torch.Size([24])
5.155452251434326 4
3776
torch.Size([13, 1, 1])
torch.Size([24, 98])
tensor([0.5936, 0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
7.045142650604248 6
torch.Size([13, 1, 1])
torch.Size([24, 98])
tensor([-0.4517,  0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-2.5423], device='cuda:0')
size torch.Size([24])
7.603046417236328 7
torch.Size([13, 1, 1])
torch.Size([24, 98])
tensor([-1.4970,  0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936, 0.5936], device='cuda:0')
size torch.Size([24])
6.074561595916748 7
torch.Size([13, 1, 1])
torch.Size([24, 98])
tensor([], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-2.5423], device='cuda:0')
size torch.Size([24])
9.386470794677734 8
torch.Size([13, 1, 1])
torch.Size([24, 98])
tensor([0.5936, 0.5936], device='cuda:0')
size torc

In [15]:

import models.InformerAutoregressiveFullRegression as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("informer_regression_eicu2.pytorch"))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
    loss = 0
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        print(output.size())
        a = outpu.item()
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        
        prediction_sofa_difference = (a-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        print(a, sum(b))
        loss_term = (outpu.squeeze()-sum(b))**2
        loss+=loss_term
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FN += 1
        else:
            if prediction_sofa_difference: FP +=1
            else: TN += 1
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()

    test_loss+=loss

print(TP, FP, FN, TN)
print()
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
print(accuracy_sepsis)

[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    1.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.   

  0%|          | 0/119 [00:00<?, ?it/s]

0
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([ 0.5936,  0.5936,  0.5936, -0.4517, -0.4517, -0.4517,  0.5936],
       device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-0.4517,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936], device='cuda:0')
size torch.Size([24])
6.426902770996094 8
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([ 0.5936,  0.5936,  0.5936,  0.5936, -0.4517, -0.4517, -0.4517, -0.4517,
        -0.4517, -0.4517, -0.4517, -0.4517, -0.4517, -0.4517, -0.4517,  0.5936,
        -0.4517, -0.4517, -0.4517, -0.4517, -0.4517, -0.4517], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-0.4517, -0.4517, -0.4517,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,
         0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,
         0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936],
       device='cuda:0')
size torch.Size([24])
5.808561325073242 7
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([], de

  1%|          | 1/119 [00:00<00:52,  2.23it/s]

size torch.Size([24])
6.815213680267334 6
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936,
        0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936,
        0.5936, 0.5936], device='cuda:0')
size torch.Size([24])
6.111606121063232 5
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([ 0.5936,  0.5936,  0.5936,  0.5936,  0.5936,  0.5936, -0.4517,  0.5936],
       device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936,
        0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936],
       device='cuda:0')
size torch.Size([24])
10.06225872039795 12
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([0.5936, 0.5936, 0.5936, 0.5936, 0.5936, 0.5936], device='cuda:0')
size torch.Size([24])

...

size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
6.538900375366211 5
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([-2.5423, -2.5423, -2.5423, -2.5423, -2.5423, -2.5423], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([ 0.5936, -2.5423, -2.5423], device='cuda:0')
size torch.Size([24])
9.31406307220459 11
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([-0.4517, -0.4517, -0.4517], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-2.5423,  0.5936, -0.4517, -0.4517], device='cuda:0')
size torch.Size([24])
8.600301742553711 9
3744
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([ 0.5936, -2.5423, -2.5423], device='cuda:0')
size torch.Size([24])
8.090353012084961 4
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936], device='cuda:0')
size t

 99%|█████████▉| 118/119 [00:51<00:00,  2.02it/s]

6.9656500816345215 5
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
5.7058305740356445 5
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([-1.4970, -1.4970, -1.4970], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-2.5423], device='cuda:0')
size torch.Size([24])
7.867068767547607 7
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([], device='cuda:0')
size torch.Size([24])
6.968908786773682 6
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-1.4970], device='cuda:0')
size torch.Size([24])
6.679592609405518 6
torch.Size([32, 1, 1])
torch.Size([24, 98])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([ 0.5936,  0.5936, -0.4517,  0.5936], devi

100%|██████████| 119/119 [00:51<00:00,  2.31it/s]

size torch.Size([24])
7.120606899261475 6
torch.Size([13, 1, 1])
torch.Size([24, 98])
tensor([-0.4517,  0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-2.5423], device='cuda:0')
size torch.Size([24])
7.2276129722595215 7
torch.Size([13, 1, 1])
torch.Size([24, 98])
tensor([-1.4970,  0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([0.5936, 0.5936], device='cuda:0')
size torch.Size([24])
6.517602443695068 7
torch.Size([13, 1, 1])
torch.Size([24, 98])
tensor([], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-2.5423], device='cuda:0')
size torch.Size([24])
10.38204288482666 8
torch.Size([13, 1, 1])
torch.Size([24, 98])
tensor([0.5936, 0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
tensor([-0.4517, -0.4517], device='cuda:0')
size torch.Size([24])
8.733040809631348 9
torch.Size([13, 1, 1])
torch.Size([24, 98])
tensor([0.5936], device='cuda:0')
size torch.Size([24])
torch.Size([24, 196])
t

# 9 Sofa Analysis IMS Informer Detach

# Autof SOFA Analysis Autoformer

# DLinear SOFA Analysis DLINEAR

# Linear SOFA Analysis Linear

# 1 STRATS DMS

# 2 STRATS IMS Teacher Forcing

# 4 STRATS IMS Backprop

# 3 STRATS IMS Detach

# Informer with Attention-DMS

# NEW EXPERIMENTS